In [1]:
ls

ATLAS_R2.0/            Processed_data_nii/
Dataset_Brain_lesion/  datafile/
Dataset_Cardiac/       datapreprocessing_ATLAS.ipynb*
Dataset_Prostate/      datapreprocessing_MnMCardiac.ipynb*
OpenDataset/           datapreprocessing_Prostate.ipynb*


In [2]:
import glob
import random
import SimpleITK as sitk
import numpy as np
import os

In [3]:
datadownloadpath = './Processed_data_nii/'

In [4]:
imgs = glob.glob(datadownloadpath + "/*/Case??.nii.gz")
imgs = [ fid for fid in sorted(imgs) ]
segs = glob.glob(datadownloadpath + "/*/Case??_*egmentation.nii.gz")
segs = [ fid for fid in sorted(segs) ]

pids = [pid.split("/")[-1].split(".")[0].split("_")[0] for pid in imgs]

In [5]:
imgs

['./Processed_data_nii/BIDMC/Case00.nii.gz',
 './Processed_data_nii/BIDMC/Case02.nii.gz',
 './Processed_data_nii/BIDMC/Case03.nii.gz',
 './Processed_data_nii/BIDMC/Case04.nii.gz',
 './Processed_data_nii/BIDMC/Case05.nii.gz',
 './Processed_data_nii/BIDMC/Case06.nii.gz',
 './Processed_data_nii/BIDMC/Case07.nii.gz',
 './Processed_data_nii/BIDMC/Case08.nii.gz',
 './Processed_data_nii/BIDMC/Case09.nii.gz',
 './Processed_data_nii/BIDMC/Case10.nii.gz',
 './Processed_data_nii/BIDMC/Case11.nii.gz',
 './Processed_data_nii/BIDMC/Case12.nii.gz',
 './Processed_data_nii/BMC/Case00.nii.gz',
 './Processed_data_nii/BMC/Case01.nii.gz',
 './Processed_data_nii/BMC/Case02.nii.gz',
 './Processed_data_nii/BMC/Case03.nii.gz',
 './Processed_data_nii/BMC/Case04.nii.gz',
 './Processed_data_nii/BMC/Case05.nii.gz',
 './Processed_data_nii/BMC/Case06.nii.gz',
 './Processed_data_nii/BMC/Case07.nii.gz',
 './Processed_data_nii/BMC/Case08.nii.gz',
 './Processed_data_nii/BMC/Case09.nii.gz',
 './Processed_data_nii/BMC/Cas

In [6]:
segs

['./Processed_data_nii/BIDMC/Case00_segmentation.nii.gz',
 './Processed_data_nii/BIDMC/Case02_segmentation.nii.gz',
 './Processed_data_nii/BIDMC/Case03_segmentation.nii.gz',
 './Processed_data_nii/BIDMC/Case04_segmentation.nii.gz',
 './Processed_data_nii/BIDMC/Case05_segmentation.nii.gz',
 './Processed_data_nii/BIDMC/Case06_segmentation.nii.gz',
 './Processed_data_nii/BIDMC/Case07_segmentation.nii.gz',
 './Processed_data_nii/BIDMC/Case08_segmentation.nii.gz',
 './Processed_data_nii/BIDMC/Case09_segmentation.nii.gz',
 './Processed_data_nii/BIDMC/Case10_segmentation.nii.gz',
 './Processed_data_nii/BIDMC/Case11_segmentation.nii.gz',
 './Processed_data_nii/BIDMC/Case12_segmentation.nii.gz',
 './Processed_data_nii/BMC/Case00_Segmentation.nii.gz',
 './Processed_data_nii/BMC/Case01_Segmentation.nii.gz',
 './Processed_data_nii/BMC/Case02_Segmentation.nii.gz',
 './Processed_data_nii/BMC/Case03_Segmentation.nii.gz',
 './Processed_data_nii/BMC/Case04_Segmentation.nii.gz',
 './Processed_data_nii/B

In [7]:
pids

['Case00',
 'Case02',
 'Case03',
 'Case04',
 'Case05',
 'Case06',
 'Case07',
 'Case08',
 'Case09',
 'Case10',
 'Case11',
 'Case12',
 'Case00',
 'Case01',
 'Case02',
 'Case03',
 'Case04',
 'Case05',
 'Case06',
 'Case07',
 'Case08',
 'Case09',
 'Case10',
 'Case11',
 'Case12',
 'Case13',
 'Case14',
 'Case15',
 'Case16',
 'Case17',
 'Case18',
 'Case19',
 'Case20',
 'Case21',
 'Case22',
 'Case23',
 'Case24',
 'Case25',
 'Case26',
 'Case27',
 'Case28',
 'Case29',
 'Case38',
 'Case39',
 'Case40',
 'Case41',
 'Case42',
 'Case43',
 'Case44',
 'Case45',
 'Case46',
 'Case47',
 'Case48',
 'Case49',
 'Case00',
 'Case01',
 'Case02',
 'Case03',
 'Case04',
 'Case05',
 'Case06',
 'Case07',
 'Case08',
 'Case09',
 'Case10',
 'Case11',
 'Case12',
 'Case13',
 'Case14',
 'Case15',
 'Case16',
 'Case17',
 'Case18',
 'Case00',
 'Case01',
 'Case02',
 'Case03',
 'Case04',
 'Case05',
 'Case06',
 'Case07',
 'Case08',
 'Case09',
 'Case10',
 'Case11',
 'Case12',
 'Case13',
 'Case14',
 'Case15',
 'Case16',
 'Case17',

In [8]:
# helper functions copy pasted
def resample_by_res(mov_img_obj, new_spacing, interpolator = sitk.sitkLinear, logging = True):
    resample = sitk.ResampleImageFilter()
    resample.SetInterpolator(interpolator)
    resample.SetOutputDirection(mov_img_obj.GetDirection())
    resample.SetOutputOrigin(mov_img_obj.GetOrigin())
    resample.SetUseNearestNeighborExtrapolator(True)
    mov_spacing = mov_img_obj.GetSpacing()

    resample.SetOutputSpacing(new_spacing)
    RES_COE = np.array(mov_spacing) * 1.0 / np.array(new_spacing)
    new_size = np.array(mov_img_obj.GetSize()) *  RES_COE 

    resample.SetSize( [int(sz+1) for sz in new_size] )
    if logging:
        print("Spacing: {} -> {}".format(mov_spacing, new_spacing))
        print("Size {} -> {}".format( mov_img_obj.GetSize(), new_size ))

    return resample.Execute(mov_img_obj)

def resample_lb_by_res(mov_lb_obj, new_spacing, interpolator = sitk.sitkLinear, ref_img = None, logging = True):
    src_mat = sitk.GetArrayFromImage(mov_lb_obj)
    lbvs = np.unique(src_mat)
    if logging:
        print("Label values: {}".format(lbvs))
    for idx, lbv in enumerate(lbvs):
        _src_curr_mat = np.float32(src_mat == lbv) 
        _src_curr_obj = sitk.GetImageFromArray(_src_curr_mat)
        _src_curr_obj.CopyInformation(mov_lb_obj)
        _tar_curr_obj = resample_by_res( _src_curr_obj, new_spacing, interpolator, logging )
        _tar_curr_mat = np.rint(sitk.GetArrayFromImage(_tar_curr_obj)) * lbv
        if idx == 0:
            out_vol = _tar_curr_mat
        else:
            out_vol[_tar_curr_mat == lbv] = lbv
    out_obj = sitk.GetImageFromArray(out_vol)
    out_obj.SetSpacing( _tar_curr_obj.GetSpacing() )
    if ref_img != None:
        out_obj.CopyInformation(ref_img)
    return out_obj

## Step 1, Data resample

In [4]:
savefold = './Dataset_Prostate/'
if os.path.exists(savefold) == False:
    os.mkdir(savefold)

In [10]:
targetspacing = [0.8, 0.8, 1.5]

In [11]:
for (imgfile, lblfile, pid) in zip(imgs, segs, pids):
    
    saving_group = savefold + str(imgfile.split('/')[-2])
    if os.path.exists(saving_group) == False:
        os.mkdir(saving_group)
    
    saving_pid_group = saving_group + '/' + pid
    if os.path.exists(saving_pid_group) == False:
        os.mkdir(saving_pid_group)
    imgsavepath = saving_pid_group + '/image.nii.gz'
    lblsavepath = saving_pid_group + '/seg.nii.gz'
    
    img_obj = sitk.ReadImage( imgfile )
    seg_obj = sitk.ReadImage( lblfile )
    img_spa_ori = img_obj.GetSpacing()
    seg_obj.SetSpacing(img_spa_ori)
    res_img_o = resample_by_res(img_obj, targetspacing, interpolator = sitk.sitkLinear,
                                    logging = True)
    res_lb_o = resample_lb_by_res(seg_obj, targetspacing, interpolator = sitk.sitkLinear,
                                  ref_img = res_img_o, logging = True)
    sitk.WriteImage(res_img_o, imgsavepath, True) 
    sitk.WriteImage(res_lb_o, lblsavepath, True) 

Spacing: (0.3645839989185333, 0.3645826578140259, 2.200010061264038) -> [0.8, 0.8, 1.5]
Size (384, 384, 47) -> [175.00031948 174.99967575  68.93364859]
Label values: [0. 1.]
Spacing: (0.3645839989185333, 0.3645826578140259, 2.200010061264038) -> [0.8, 0.8, 1.5]
Size (384, 384, 47) -> [175.00031948 174.99967575  68.93364859]
Spacing: (0.3645839989185333, 0.3645826578140259, 2.200010061264038) -> [0.8, 0.8, 1.5]
Size (384, 384, 47) -> [175.00031948 174.99967575  68.93364859]
Spacing: (0.3645839989185333, 0.3645866811275482, 2.1999800205230713) -> [0.8, 0.8, 1.5]
Size (384, 384, 54) -> [175.00031948 175.00160694  79.19928074]
Label values: [0. 1.]
Spacing: (0.3645839989185333, 0.3645866811275482, 2.1999800205230713) -> [0.8, 0.8, 1.5]
Size (384, 384, 54) -> [175.00031948 175.00160694  79.19928074]
Spacing: (0.3645839989185333, 0.3645866811275482, 2.1999800205230713) -> [0.8, 0.8, 1.5]
Size (384, 384, 54) -> [175.00031948 175.00160694  79.19928074]
Spacing: (0.36458534002304077, 0.36458399

Spacing: (0.4166666865348816, 0.4166666865348816, 4.000000476837158) -> [0.8, 0.8, 1.5]
Size (384, 384, 23) -> [200.00000954 200.00000954  61.33334064]
Label values: [0. 1. 2.]
Spacing: (0.4166666865348816, 0.4166666865348816, 4.000000476837158) -> [0.8, 0.8, 1.5]
Size (384, 384, 23) -> [200.00000954 200.00000954  61.33334064]
Spacing: (0.4166666865348816, 0.4166666865348816, 4.000000476837158) -> [0.8, 0.8, 1.5]
Size (384, 384, 23) -> [200.00000954 200.00000954  61.33334064]
Spacing: (0.4166666865348816, 0.4166666865348816, 4.000000476837158) -> [0.8, 0.8, 1.5]
Size (384, 384, 23) -> [200.00000954 200.00000954  61.33334064]
Spacing: (0.4166666865348816, 0.4166666865348816, 2.99999737739563) -> [0.8, 0.8, 1.5]
Size (384, 384, 32) -> [200.00000954 200.00000954  63.99994405]
Label values: [0. 1. 2.]
Spacing: (0.4166666865348816, 0.4166666865348816, 2.99999737739563) -> [0.8, 0.8, 1.5]
Size (384, 384, 32) -> [200.00000954 200.00000954  63.99994405]
Spacing: (0.4166666865348816, 0.41666668

Spacing: (0.5208333134651184, 0.5208333134651184, 3.5999999046325684) -> [0.8, 0.8, 1.5]
Size (384, 384, 24) -> [249.99999046 249.99999046  57.59999847]
Label values: [0. 1.]
Spacing: (0.5208333134651184, 0.5208333134651184, 3.5999999046325684) -> [0.8, 0.8, 1.5]
Size (384, 384, 24) -> [249.99999046 249.99999046  57.59999847]
Spacing: (0.5208333134651184, 0.5208333134651184, 3.5999999046325684) -> [0.8, 0.8, 1.5]
Size (384, 384, 24) -> [249.99999046 249.99999046  57.59999847]
Spacing: (0.5208333134651184, 0.5208333134651184, 3.5999999046325684) -> [0.8, 0.8, 1.5]
Size (384, 384, 24) -> [249.99999046 249.99999046  57.59999847]
Label values: [0. 1.]
Spacing: (0.5208333134651184, 0.5208333134651184, 3.5999999046325684) -> [0.8, 0.8, 1.5]
Size (384, 384, 24) -> [249.99999046 249.99999046  57.59999847]
Spacing: (0.5208333134651184, 0.5208333134651184, 3.5999999046325684) -> [0.8, 0.8, 1.5]
Size (384, 384, 24) -> [249.99999046 249.99999046  57.59999847]
Spacing: (0.5208333134651184, 0.520833

Label values: [0. 1.]
Spacing: (0.5342610478401184, 0.42852187156677246, 1.2500154972076416) -> [0.8, 0.8, 1.5]
Size (384, 384, 64) -> [256.44530296 205.69049835  53.33399455]
Spacing: (0.5342610478401184, 0.42852187156677246, 1.2500154972076416) -> [0.8, 0.8, 1.5]
Size (384, 384, 64) -> [256.44530296 205.69049835  53.33399455]
Spacing: (0.5342610478401184, 0.4388573169708252, 1.2499207258224487) -> [0.8, 0.8, 1.5]
Size (384, 384, 64) -> [256.44530296 210.65151215  53.32995097]
Label values: [0. 1.]
Spacing: (0.5342610478401184, 0.4388573169708252, 1.2499207258224487) -> [0.8, 0.8, 1.5]
Size (384, 384, 64) -> [256.44530296 210.65151215  53.32995097]
Spacing: (0.5342610478401184, 0.4388573169708252, 1.2499207258224487) -> [0.8, 0.8, 1.5]
Size (384, 384, 64) -> [256.44530296 210.65151215  53.32995097]
Spacing: (0.5342610478401184, 0.4388573169708252, 1.2498712539672852) -> [0.8, 0.8, 1.5]
Size (384, 384, 64) -> [256.44530296 210.65151215  53.32784017]
Label values: [0. 1.]
Spacing: (0.53

Spacing: (0.5208333134651184, 0.5208333134651184, 3.6000022888183594) -> [0.8, 0.8, 1.5]
Size (384, 384, 20) -> [249.99999046 249.99999046  48.00003052]
Label values: [0. 1. 2.]
Spacing: (0.5208333134651184, 0.5208333134651184, 3.6000022888183594) -> [0.8, 0.8, 1.5]
Size (384, 384, 20) -> [249.99999046 249.99999046  48.00003052]
Spacing: (0.5208333134651184, 0.5208333134651184, 3.6000022888183594) -> [0.8, 0.8, 1.5]
Size (384, 384, 20) -> [249.99999046 249.99999046  48.00003052]
Spacing: (0.5208333134651184, 0.5208333134651184, 3.6000022888183594) -> [0.8, 0.8, 1.5]
Size (384, 384, 20) -> [249.99999046 249.99999046  48.00003052]
Spacing: (0.5208333134651184, 0.5208333134651184, 3.6000008583068848) -> [0.8, 0.8, 1.5]
Size (384, 384, 20) -> [249.99999046 249.99999046  48.00001144]
Label values: [0. 1. 2.]
Spacing: (0.5208333134651184, 0.5208333134651184, 3.6000008583068848) -> [0.8, 0.8, 1.5]
Size (384, 384, 20) -> [249.99999046 249.99999046  48.00001144]
Spacing: (0.5208333134651184, 0.

Spacing: (0.5208333134651184, 0.5208333134651184, 3.5999984741210938) -> [0.8, 0.8, 1.5]
Size (384, 384, 20) -> [249.99999046 249.99999046  47.99997965]
Label values: [0. 1. 2.]
Spacing: (0.5208333134651184, 0.5208333134651184, 3.5999984741210938) -> [0.8, 0.8, 1.5]
Size (384, 384, 20) -> [249.99999046 249.99999046  47.99997965]
Spacing: (0.5208333134651184, 0.5208333134651184, 3.5999984741210938) -> [0.8, 0.8, 1.5]
Size (384, 384, 20) -> [249.99999046 249.99999046  47.99997965]
Spacing: (0.5208333134651184, 0.5208333134651184, 3.5999984741210938) -> [0.8, 0.8, 1.5]
Size (384, 384, 20) -> [249.99999046 249.99999046  47.99997965]
Spacing: (0.5, 0.5, 4.000001907348633) -> [0.8, 0.8, 1.5]
Size (384, 384, 18) -> [240.         240.          48.00002289]
Label values: [0. 2.]
Spacing: (0.5, 0.5, 4.000001907348633) -> [0.8, 0.8, 1.5]
Size (384, 384, 18) -> [240.         240.          48.00002289]
Spacing: (0.5, 0.5, 4.000001907348633) -> [0.8, 0.8, 1.5]
Size (384, 384, 18) -> [240.         24

Spacing: (0.5208333134651184, 0.5208333134651184, 3.299999952316284) -> [0.8, 0.8, 1.5]
Size (384, 384, 23) -> [249.99999046 249.99999046  50.59999927]
Label values: [0. 1.]
Spacing: (0.5208333134651184, 0.5208333134651184, 3.299999952316284) -> [0.8, 0.8, 1.5]
Size (384, 384, 23) -> [249.99999046 249.99999046  50.59999927]
Spacing: (0.5208333134651184, 0.5208333134651184, 3.299999952316284) -> [0.8, 0.8, 1.5]
Size (384, 384, 23) -> [249.99999046 249.99999046  50.59999927]
Spacing: (0.5208333134651184, 0.5208333134651184, 3.299999952316284) -> [0.8, 0.8, 1.5]
Size (384, 384, 23) -> [249.99999046 249.99999046  50.59999927]
Label values: [0. 1.]
Spacing: (0.5208333134651184, 0.5208333134651184, 3.299999952316284) -> [0.8, 0.8, 1.5]
Size (384, 384, 23) -> [249.99999046 249.99999046  50.59999927]
Spacing: (0.5208333134651184, 0.5208333134651184, 3.299999952316284) -> [0.8, 0.8, 1.5]
Size (384, 384, 23) -> [249.99999046 249.99999046  50.59999927]
Spacing: (0.5208333134651184, 0.520833313465

## Step 2, Intensity Normalization and Binary Segmentation

In [12]:
# get the volume list
imgs_volume = glob.glob(savefold + "*/*/image.nii.gz")
imgs_volume = [ fid for fid in sorted(imgs_volume) ]

segs_volume = glob.glob(savefold + "*/*/seg.nii.gz")
segs_volume = [ fid for fid in sorted(segs_volume) ]

pids_volume = [pids_volume.split("/")[-2] for pids_volume in imgs_volume]

In [13]:
imgs_volume

['./Dataset_Prostate/BIDMC/Case00/image.nii.gz',
 './Dataset_Prostate/BIDMC/Case02/image.nii.gz',
 './Dataset_Prostate/BIDMC/Case03/image.nii.gz',
 './Dataset_Prostate/BIDMC/Case04/image.nii.gz',
 './Dataset_Prostate/BIDMC/Case05/image.nii.gz',
 './Dataset_Prostate/BIDMC/Case06/image.nii.gz',
 './Dataset_Prostate/BIDMC/Case07/image.nii.gz',
 './Dataset_Prostate/BIDMC/Case08/image.nii.gz',
 './Dataset_Prostate/BIDMC/Case09/image.nii.gz',
 './Dataset_Prostate/BIDMC/Case10/image.nii.gz',
 './Dataset_Prostate/BIDMC/Case11/image.nii.gz',
 './Dataset_Prostate/BIDMC/Case12/image.nii.gz',
 './Dataset_Prostate/BMC/Case00/image.nii.gz',
 './Dataset_Prostate/BMC/Case01/image.nii.gz',
 './Dataset_Prostate/BMC/Case02/image.nii.gz',
 './Dataset_Prostate/BMC/Case03/image.nii.gz',
 './Dataset_Prostate/BMC/Case04/image.nii.gz',
 './Dataset_Prostate/BMC/Case05/image.nii.gz',
 './Dataset_Prostate/BMC/Case06/image.nii.gz',
 './Dataset_Prostate/BMC/Case07/image.nii.gz',
 './Dataset_Prostate/BMC/Case08/imag

In [14]:
segs_volume

['./Dataset_Prostate/BIDMC/Case00/seg.nii.gz',
 './Dataset_Prostate/BIDMC/Case02/seg.nii.gz',
 './Dataset_Prostate/BIDMC/Case03/seg.nii.gz',
 './Dataset_Prostate/BIDMC/Case04/seg.nii.gz',
 './Dataset_Prostate/BIDMC/Case05/seg.nii.gz',
 './Dataset_Prostate/BIDMC/Case06/seg.nii.gz',
 './Dataset_Prostate/BIDMC/Case07/seg.nii.gz',
 './Dataset_Prostate/BIDMC/Case08/seg.nii.gz',
 './Dataset_Prostate/BIDMC/Case09/seg.nii.gz',
 './Dataset_Prostate/BIDMC/Case10/seg.nii.gz',
 './Dataset_Prostate/BIDMC/Case11/seg.nii.gz',
 './Dataset_Prostate/BIDMC/Case12/seg.nii.gz',
 './Dataset_Prostate/BMC/Case00/seg.nii.gz',
 './Dataset_Prostate/BMC/Case01/seg.nii.gz',
 './Dataset_Prostate/BMC/Case02/seg.nii.gz',
 './Dataset_Prostate/BMC/Case03/seg.nii.gz',
 './Dataset_Prostate/BMC/Case04/seg.nii.gz',
 './Dataset_Prostate/BMC/Case05/seg.nii.gz',
 './Dataset_Prostate/BMC/Case06/seg.nii.gz',
 './Dataset_Prostate/BMC/Case07/seg.nii.gz',
 './Dataset_Prostate/BMC/Case08/seg.nii.gz',
 './Dataset_Prostate/BMC/Case09

In [15]:
pids_volume

['Case00',
 'Case02',
 'Case03',
 'Case04',
 'Case05',
 'Case06',
 'Case07',
 'Case08',
 'Case09',
 'Case10',
 'Case11',
 'Case12',
 'Case00',
 'Case01',
 'Case02',
 'Case03',
 'Case04',
 'Case05',
 'Case06',
 'Case07',
 'Case08',
 'Case09',
 'Case10',
 'Case11',
 'Case12',
 'Case13',
 'Case14',
 'Case15',
 'Case16',
 'Case17',
 'Case18',
 'Case19',
 'Case20',
 'Case21',
 'Case22',
 'Case23',
 'Case24',
 'Case25',
 'Case26',
 'Case27',
 'Case28',
 'Case29',
 'Case38',
 'Case39',
 'Case40',
 'Case41',
 'Case42',
 'Case43',
 'Case44',
 'Case45',
 'Case46',
 'Case47',
 'Case48',
 'Case49',
 'Case00',
 'Case01',
 'Case02',
 'Case03',
 'Case04',
 'Case05',
 'Case06',
 'Case07',
 'Case08',
 'Case09',
 'Case10',
 'Case11',
 'Case12',
 'Case13',
 'Case14',
 'Case15',
 'Case16',
 'Case17',
 'Case18',
 'Case00',
 'Case01',
 'Case02',
 'Case03',
 'Case04',
 'Case05',
 'Case06',
 'Case07',
 'Case08',
 'Case09',
 'Case10',
 'Case11',
 'Case12',
 'Case13',
 'Case14',
 'Case15',
 'Case16',
 'Case17',

In [16]:
for (imgfile, segfile, pid) in zip(imgs_volume, segs_volume, pids_volume):
    img_obj = sitk.ReadImage( imgfile )
    seg_obj = sitk.ReadImage( segfile )
    
    array = sitk.GetArrayFromImage(img_obj)

    pixel_mean = np.mean(array)
    pixel_std = np.std(array)
    array = (array - pixel_mean) / pixel_std

    normalized_img = sitk.GetImageFromArray(array)
    normalized_img.CopyInformation(img_obj)
    
    array = sitk.GetArrayFromImage(seg_obj)

    array[array > 0] = 1

    binary_seg = sitk.GetImageFromArray(array)
    binary_seg.CopyInformation(seg_obj)
    
    sitk.WriteImage(normalized_img, imgfile, True)
    sitk.WriteImage(binary_seg, segfile, True)
    print('Id ' + pid + ' normalized')

Id Case00 normalized
Id Case02 normalized
Id Case03 normalized
Id Case04 normalized
Id Case05 normalized
Id Case06 normalized
Id Case07 normalized
Id Case08 normalized
Id Case09 normalized
Id Case10 normalized
Id Case11 normalized
Id Case12 normalized
Id Case00 normalized
Id Case01 normalized
Id Case02 normalized
Id Case03 normalized
Id Case04 normalized
Id Case05 normalized
Id Case06 normalized
Id Case07 normalized
Id Case08 normalized
Id Case09 normalized
Id Case10 normalized
Id Case11 normalized
Id Case12 normalized
Id Case13 normalized
Id Case14 normalized
Id Case15 normalized
Id Case16 normalized
Id Case17 normalized
Id Case18 normalized
Id Case19 normalized
Id Case20 normalized
Id Case21 normalized
Id Case22 normalized
Id Case23 normalized
Id Case24 normalized
Id Case25 normalized
Id Case26 normalized
Id Case27 normalized
Id Case28 normalized
Id Case29 normalized
Id Case38 normalized
Id Case39 normalized
Id Case40 normalized
Id Case41 normalized
Id Case42 normalized
Id Case43 nor

## Step 3, Generate datafiles.

In [3]:
datafilepath = './datafile/Dataset_Prostate/'
training_folder = 'BMC'
if os.path.exists(datafilepath) == False:
    os.mkdir(datafilepath)

In [5]:
domainlists = glob.glob(savefold + '*/')

In [6]:
domainlists

['./Dataset_Prostate/HK/',
 './Dataset_Prostate/UCL/',
 './Dataset_Prostate/BMC/',
 './Dataset_Prostate/I2CVB/',
 './Dataset_Prostate/BIDMC/',
 './Dataset_Prostate/RUNMC/']

In [7]:
# just utilize the generated datafiles, or randomize val/test like this.

In [8]:
src_valfile = datafilepath + training_folder + 'val'
src_testfile = datafilepath + training_folder + 'test'
#
Imgdatafiles_val = src_valfile + '/Imgpre-eval.txt'
Lbldatafiles_val = src_valfile + '/seg-eval.txt'
#
Imgdatafiles_test = src_testfile + '/Imgpre-eval.txt'
Lbldatafiles_test = src_testfile + '/seg-eval.txt'

In [9]:
with open(Imgdatafiles_val) as f:
    lines_imgval = f.readlines()
with open(Lbldatafiles_val) as f:
    lines_lblval = f.readlines()
with open(Imgdatafiles_test) as f:
    lines_imgtest = f.readlines()
with open(Lbldatafiles_test) as f:
    lines_lbltest = f.readlines()

In [10]:
lines_img = lines_imgval + lines_imgtest
lines_lbl = lines_lblval + lines_lbltest
lines_img_sorted = sorted(lines_img)
lines_lbl_sorted = sorted(lines_lbl)

In [11]:
random.seed(0)
rndlist = list(range(len(lines_img)))
random.shuffle(rndlist)
lines_img = []
lines_lbl = []
for listid in rndlist:
    lines_img.append(lines_img_sorted[listid])
    lines_lbl.append(lines_lbl_sorted[listid])

In [12]:
# write image files
with open(Imgdatafiles_val, 'w') as f:
    for filename in lines_img[:int(len(lines_img)/3)]: 
        f.write(filename)
# write label files
with open(Lbldatafiles_val, 'w') as f:
    for filename in lines_lbl[:int(len(lines_lbl)/3)]: 
        f.write(filename)

# write image files
with open(Imgdatafiles_test, 'w') as f:
    for filename in lines_img[int(len(lines_img)/3):]: 
        f.write(filename)
# write label files
with open(Lbldatafiles_test, 'w') as f:
    for filename in lines_lbl[int(len(lines_lbl)/3):]: 
        f.write(filename)